1. 预处理训练集，将训练集的猫与狗分为训练集和测试集，分别存放到对应的目录中

创建目录

In [1]:
import os,shutil

data_set_dir = './data_set'

train_dir = './train'

validation_dir = './validation'

test_dir = './test'

train_cats_dir = os.path.join(train_dir,'cat')

train_dogs_dir = os.path.join(train_dir,'dog')

cats_validation_dir = os.path.join(validation_dir,'cat')

dogs_validation_dir = os.path.join(validation_dir,'dog')




In [ ]:
os.mkdir(train_dir)

os.mkdir(test_dir)

os.mkdir(validation_dir)

os.mkdir(train_cats_dir)

os.mkdir(train_dogs_dir)

os.mkdir(cats_validation_dir)

os.mkdir(dogs_validation_dir)

训练数据迁移  划分 前 10000个为训练集 后2500个为测试集

In [ ]:
def data_set_build(type,target_dir):

    file_names = ['{}.{}.jpg'.format(type,i) for i in range (8000)]

    for file_name in file_names :
        src = os.path.join(data_set_dir,file_name)
        dst = os.path.join(target_dir,file_name)
        shutil.copyfile(src,dst)

    file_names = ['{}.{}.jpg'.format(type,i) for i in range (8000,10000)]

    for file_name in file_names :
       src = os.path.join(data_set_dir,file_name)
       dst = os.path.join(validation_dir,type)
       dst = os.path.join(dst,file_name)
       shutil.copyfile(src,dst)

    file_names = ['{}.{}.jpg'.format(type,i) for i in range (10000,12500)]

    for file_name in file_names :
        src = os.path.join(data_set_dir,file_name)
        dst = os.path.join(test_dir,file_name)
        shutil.copyfile(src,dst)

data_set_build('cat',train_cats_dir)
data_set_build('dog',train_dogs_dir)

导包

In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, Add, GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam


普通卷积模型

In [ ]:
def cnn_model():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(223, 223, 3)),
        tf.keras.layers.MaxPooling2D((2, 2)),
        
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),

        tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),

        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    return model

基于Resnet网络架构构建网络模型

In [4]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
def resnet_model():
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    for layer in base_model.layers:
        layer.trainable = False
    return model


准备开始模型训练

In [6]:

# input_shape = (223, 223, 3)
# num_classes = 2
# model = resnet_model(input_shape, num_classes)

model = resnet_model()

model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 230, 230,  │          0 │ input_layer_1[0]… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 56, 56,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 56, 56,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 56, 56,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 56, 56,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_3_c

 Total params: 25,686,913 (97.99 MB)

 Trainable params: 2,099,201 (8.01 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

配置训练器

In [7]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


数据预处理

In [9]:
train_image = ImageDataGenerator(rescale=1./255)
test_image = ImageDataGenerator(rescale=1./255)

train_generator = train_image.flow_from_directory(train_dir,
                                             target_size=(224,224),
                                             batch_size=32,
                                             class_mode='binary')

validation_generator = test_image.flow_from_directory(validation_dir,
                                                 target_size=(224,224),
                                                 batch_size=32,
                                                  class_mode='binary')



Found 16000 images belonging to 2 classes.
Found 4000 images belonging to 2 classes.


训练网络模型

In [10]:
history = model.fit(train_generator, batch_size=32, epochs=10, validation_data=validation_generator,
                    validation_steps=32)

Epoch 1/10


d:\Programing\Dependency\Anaconda\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


269/500 ━━━━━━━━━━━━━━━━━━━━ 3:49 992ms/step - accuracy: 0.5635 - loss: 0.7443

KeyboardInterrupt: 